In [38]:
""" File to graph several signals using the Fourier Transform
    and Fourier Coefficients
    Author: Zander Blasingame
    Course: EE 321 """

import numpy as np
from scipy import signal
import plotly.plotly as py
import plotly.graph_objs as go

In [40]:
""" Function Definitions """

# Constants
N = 2000
SIGNAL_RANGE = 40
RESOLUTION = 100E-3  # 100ms

# define discrete unit impulse
d = lambda x: 1 if x == 0 else 0


# Construct signal from fourier coefficients
def construct_signal(X_k, period=2*np.pi, N=N):
    return lambda x: np.sum(X_k(k) * np.exp(2j*np.pi*k*x/period)
                            for k in range(-N, N, 1))


def mean_squared_error(signal_1, signal_2, N=N):
    return 1/N * np.sum(np.abs(signal_1(n) - signal_2(n))**2
                        for n in range(1, N))

# Problems

## Problem 1a

In [3]:
%%latex
Given the following defintions for fourier coefficients, $X_k$

\[X_k = \delta(k) + \frac{1}{4}\delta(k-1) + \frac{1}{4}\delta(k+1) + \frac{1}{j2}\delta(k-2) + \frac{1}{j2}\delta(k+2)\]

<IPython.core.display.Latex object>

In [41]:
""" Problem 1a """
X_k = lambda k: d(k) + 0.25*d(k-1) + 0.25*(k+1) - 0.5j*d(k-2) + 0.5j*d(k+2)

# Construct and plot signal
x = construct_signal(X_k)
t = np.arange(-SIGNAL_RANGE/2, SIGNAL_RANGE/2, RESOLUTION)

data = [go.Scatter(x=t, y=[x(time).real for time in t])]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 1a')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='signal_1.png')

## Problem 1b

In [5]:
%%latex
Given the following defintions for fourier coefficients, $X_k$

\[X_k = \bigg\{\begin{align}
               &jk \; &|k| < 3\\
               &0 &\text{otherwise}
               \end{align}\]



<IPython.core.display.Latex object>

In [42]:
""" Problem 1b """
X_k = lambda k: 1j*k if abs(k) < 3 else 0

# Construct and plot signal
x = construct_signal(X_k)
t = np.arange(-SIGNAL_RANGE/2, SIGNAL_RANGE/2, RESOLUTION)

data = [go.Scatter(x=t, y=[x(time).real for time in t])]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 1b')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='signal_2.png')

# Problem 2

In [17]:
%%latex

Let the reconstructed signal, $\hat{x}(t)$ given $N$ components be defined as follows
\begin{equation}
\hat{x}(t) = \sum_{k=-N}^N X_ke^{\frac{j2\pi k}{P}t}
\end{equation}
Moreover, let the mean squared error between the original signal, $x(t)$ and the reconstructed signal be given by
\begin{equation}
\mathbb{E}(N) = \frac{1}{T} \int_0^T |x(t) - \hat{x}(t)|^2 dt
\end{equation}
In a discrete form this is represented as
\begin{equation}
\mathbb{E}(N) = \frac{1}{N} \sum_{n=1}^N |x(n) - \hat{x}(n)|^2
\end{equation}
Let the complex Fourier coefficients be expressed as
\[X_k = \frac{1}{T} \int_{t_0}^{t_0 + \frac{T}{2}} x(t)e^{-j\frac{2\pi kt}{P}}dt\]

<IPython.core.display.Latex object>

## Problem 2a

In [73]:
%%latex
Let $x_1(t)$ be defined as

\[x_1(t) = \sum_{n=-\infty}^{\infty} u(t-3n) - u(t - 3n - 2)\]
where $u(n)$ is the Heaviside step function. This can be rewritten as
\[x_1(t) = \sum_{n=-\infty}^{\infty} \text{rect}\bigg(\frac{t}{2} - \frac{3n + 1}{2}\bigg)\]
To find the complex Fourier coefficients look at two time ranges: [-1, 0] and [0, 2] such that
\[X_k = \frac{1}{3} \int_{-1}^{2} x_1(t)e^{-j\frac{2\pi kt}{3}}dt\]
This can be simplified to
\[X_k = \frac{1}{3}\bigg[ \int_{-1}^{0} x_1(t)e^{-j\frac{2\pi kt}{3}}dt
        + \int_{0}^{2} x_1(t)e^{-j\frac{2\pi kt}{3}} dt\bigg]\]
\[X_k = \frac{1}{3}\bigg[ \int_{-1}^{0} 0e^{-j\frac{2\pi kt}{3}}dt
        + \int_{0}^{2} 1e^{-j\frac{2\pi kt}{3}} dt\bigg]\]
\[X_k = \frac{1}{3} \int_{0}^{2} e^{-j\frac{2\pi kt}{3}}dt\]
\[X_k = \frac{j}{2\pi k}\bigg[e^{-j\frac{4\pi k}{3}} - 1 \bigg]\]


<IPython.core.display.Latex object>

In [43]:
""" Problem 2a """
# added term to avoid dividing by zero
X_k = lambda k: 1j/(2*np.pi*k + 1E-9) * (np.exp(-1j*4*np.pi*k/3) - 1) + 2/3*d(k)

# Construct and plot signal
x = construct_signal(X_k, 3)
t = np.arange(-SIGNAL_RANGE/2, SIGNAL_RANGE/2, RESOLUTION)

data = [go.Scatter(x=t, y=[x(time).real for time in t])]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 2a')

fig = go.Figure(data=data, layout=layout)


py.iplot(fig, filename='signal_3.png')


In [57]:
# ideal signal
x_ideal = lambda t: signal.square(2/3*np.pi*t, duty=2/3)/2 + 1/2
num_components = [n for n in range(1, 100, 1)]
errors = [mean_squared_error(construct_signal(X_k, 3, N=n), x_ideal, N=n) for n in num_components]

# data = [go.Scatter(x=t, y=[construct_signal(X_k, 3)(time).real - x_ideal(time) for time in t])]
data = [go.Scatter(x=num_components, y=errors)]
layout = go.Layout(xaxis=dict(title='N', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Error'),
                   title='Reconstruction Error 2a')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='rec_err_1.png')

In [101]:
%%latex
Let $x_2(t)$ be defined as

\[x_2(t) = \sum_{n=-\infty}^{\infty} \text{tri}(t-(2n+1))\]
where $\text{tri}(t)$ is the triangular pulse. This can be expressed as
\begin{equation}
\text{tri}(t) = \text{rect}(t/2)(1-|t|)
\end{equation}
The Fourier coefficients can be expressed as
\begin{equation}
X_k = \frac{1}{2} \int_{0}^{2} \text{tri}(t)\cdot e^{-j\pi kt}dt
\end{equation}
Further simplification yields the following
\begin{align*}
X_k &= \frac{1}{2}\bigg[\int_{-1}^{0} -t \cdot e^{-j\pi kt}dt
        + \int_{0}^{1} t \cdot e^{-j\pi kt}dt\bigg]
\end{align*}
By reversing the bounds of integration
\begin{align*}
X_k &= \int_0^1 t\cdot e^{-j\pi kt}dt\\
X_k &= \frac{j\pi kte^{-j\pi kt} + e^{-j\pi kt}}{\pi^2k^2}\bigg|_0^1\\
X_k &= \frac{(j\pi k + 1)e^{-j\pi k} - 1}{\pi^2k^2}
\end{align*}

<IPython.core.display.Latex object>

In [102]:
""" Problem 2b """
# X_k = lambda k: (np.exp(-1j*np.pi*k)*(1j*np.pi*k - np.exp(1j*np.pi*k) + 1)
#                  + np.exp(-2j*np.pi*k) * (np.exp(1j*np.pi*k) * (1j*np.pi*k - 1) + 1))/(2 * np.pi**2 * k**2)
    
X_k = lambda k: ((1j*np.pi*k + 1) * np.exp(-1j*np.pi*k) - 1) / (np.pi**2 * k**2)


# Construct and plot signal
x = construct_signal(X_k, 2)
t = np.arange(-SIGNAL_RANGE/2, SIGNAL_RANGE/2, RESOLUTION)

data = [go.Scatter(x=t, y=[x(time).real for time in t])]
layout = go.Layout(xaxis=dict(title='Time (s)', showticklabels=True,
                              tickmode='linear',
                              tickangle=0,
                              dtick=5),
                   yaxis=dict(title='Amplitude x(t)'),
                   title='Signal 2b')

fig = go.Figure(data=data, layout=layout)


py.iplot(fig, filename='signal_4.png')


PlotlyError: Request throttled. You've created/updated more charts than your allowed limit of 50/day. You may either wait one day or upgrade your account. Visit https://plot.ly/settings/subscription/ to upgrade.